# PHASE 2 PROJECT.
ANGUISTA MIRIAM KUPEKA
## Business Understanding
The film industry is a dynamic global market, encompassing all stages of movie production and distribution and exerting significant cultural and economic influence. As more companies venture into original content creation, understanding which films resonate with audiences is crucial. Analyzing box office trends enables newcomers to identify popular genres, profitable themes, and audience preferences, guiding strategic production choices. This data-driven approach empowers new studios to craft compelling content that aligns with viewer interests, enhancing their potential to make a lasting impact in an increasingly competitive industry.

## Business Problem
ABC Company has observed that major corporations are increasingly investing in original video content, and they’re eager to join in. They’ve decided to establish a new movie studio but lack experience in film production. Your role is to analyze current box office trends to determine which types of films are performing best. You will then translate these findings into actionable insights for the head of the new movie studio, helping guide strategic decisions on the types of films to produce.

## Objectives
1. Identify Popular Genres: Analyze box office and ratings data to find genres with high audience interest and financial success.
2. Analyze Audience Preferences: Review ratings to uncover common traits of well-received films.
3. Assess Profitability by Theme: Use revenue data to identify profitable themes and genres.
4. Understand Box Office Trends: Examine seasonal and long-term trends to guide ABC's release schedules and marketing strategy.

## Import the necessary packages.

In [1]:
# import the required packages 

import itertools
import numpy as np
import pandas as pd
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from collections import defaultdict

import pickle

## Load the necessary data and Data cleaning.

Creation of a cleaned gross budget.

In [ ]:
#Loading bom.movie_gross.csv.gz and renaming columns/droping irrelevant columns
bom = pd.read_csv('bom.movie_gross.csv.gz')
bom2 = bom.rename(columns={'title': 'movie'})
bom3 = bom2.drop(['domestic_gross', 'year'], axis=1)
bom3.head()

,movie,studio,foreign_gross
0,Toy Story 3,BV,652000000
1,Alice in Wonderland (2010),BV,691300000
2,Harry Potter and the Deathly Hallows Part 1,WB,664300000
3,Inception,WB,535700000
4,Shrek Forever After,P/DW,513900000


In [11]:
# Loading tn.movie_budgets.csv and droping irrelevant columns
tnmovie = pd.read_csv('tn.movie_budgets.csv.gz')
tnmovie1 = tnmovie.drop(['id'], axis=1)
tnmovie1.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [12]:
#Merging tnmovie1 and and bom3
gross_budget= pd.merge(bom3, tnmovie1, on='movie', how='inner')
gross_budget.head()

,movie,studio,foreign_gross,release_date,production_budget,domestic_gross,worldwide_gross
0,Toy Story 3,BV,652000000,"Jun 18, 2010","$200,000,000","$415,004,880","$1,068,879,522"
1,Inception,WB,535700000,"Jul 16, 2010","$160,000,000","$292,576,195","$835,524,642"
2,Shrek Forever After,P/DW,513900000,"May 21, 2010","$165,000,000","$238,736,787","$756,244,673"
3,The Twilight Saga: Eclipse,Sum.,398000000,"Jun 30, 2010","$68,000,000","$300,531,751","$706,102,828"
4,Iron Man 2,Par.,311500000,"May 7, 2010","$170,000,000","$312,433,331","$621,156,389"


In [13]:
# Renaming columns our for new merged data(Gross budget)
gross_budget.rename(columns={"movie":"original_title"}, inplace=True)

In [14]:
gross_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_title     1247 non-null   object
 1   studio             1246 non-null   object
 2   foreign_gross      1086 non-null   object
 3   release_date       1247 non-null   object
 4   production_budget  1247 non-null   object
 5   domestic_gross     1247 non-null   object
 6   worldwide_gross    1247 non-null   object
dtypes: object(7)
memory usage: 68.3+ KB


In [15]:
# Removing sysmbols, commas and replacing with empty strings)
for col in ['production_budget', 'domestic_gross', 'worldwide_gross']:

    gross_budget[col] = gross_budget[col].replace({'\\$': '', ',': ''}, regex=True).astype(float)
gross_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_title     1247 non-null   object 
 1   studio             1246 non-null   object 
 2   foreign_gross      1086 non-null   object 
 3   release_date       1247 non-null   object 
 4   production_budget  1247 non-null   float64
 5   domestic_gross     1247 non-null   float64
 6   worldwide_gross    1247 non-null   float64
dtypes: float64(3), object(4)
memory usage: 68.3+ KB


In [9]:
# Convert foreign gross from string to numeric
gross_budget['foreign_gross'] = pd.to_numeric(gross_budget['foreign_gross'], errors='coerce') 
gross_budget['production_budget'] = pd.to_numeric(gross_budget['production_budget'], errors='coerce')
gross_budget['domestic_gross'] = pd.to_numeric(gross_budget['domestic_gross'], errors='coerce')
gross_budget['worldwide_gross'] = pd.to_numeric(gross_budget['worldwide_gross'], errors='coerce')
gross_budget.head()

,movie,studio,foreign_gross,release_date,production_budget,domestic_gross,worldwide_gross
0,Toy Story 3,BV,652000000.0,"Jun 18, 2010",200000000.0,415004880.0,1.068880e+09
1,Inception,WB,535700000.0,"Jul 16, 2010",160000000.0,292576195.0,8.355246e+08
2,Shrek Forever After,P/DW,513900000.0,"May 21, 2010",165000000.0,238736787.0,7.562447e+08
3,The Twilight Saga: Eclipse,Sum.,398000000.0,"Jun 30, 2010",68000000.0,300531751.0,7.061028e+08
4,Iron Man 2,Par.,311500000.0,"May 7, 2010",170000000.0,312433331.0,6.211564e+08


In [10]:
gross_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              1247 non-null   object 
 1   studio             1246 non-null   object 
 2   foreign_gross      1082 non-null   float64
 3   release_date       1247 non-null   object 
 4   production_budget  1247 non-null   float64
 5   domestic_gross     1247 non-null   float64
 6   worldwide_gross    1247 non-null   float64
dtypes: float64(4), object(3)
memory usage: 68.3+ KB


In [16]:
#Check for missing values
gross_budget.isna().sum()

original_title         0
studio                 1
foreign_gross        161
release_date           0
production_budget      0
domestic_gross         0
worldwide_gross        0
dtype: int64

In [ ]:
# Drop missing values in every colum 
for column in gross_budget.columns:
    gross_budget = gross_budget.dropna(subset=[column])

gross_budget.isna().sum()

original_title       0
studio               0
foreign_gross        0
release_date         0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [18]:
#Check for duplicates
gross_budget.duplicated().sum()

0

In [19]:
gross_budget.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1085 entries, 0 to 1244
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_title     1085 non-null   object 
 1   studio             1085 non-null   object 
 2   foreign_gross      1085 non-null   object 
 3   release_date       1085 non-null   object 
 4   production_budget  1085 non-null   float64
 5   domestic_gross     1085 non-null   float64
 6   worldwide_gross    1085 non-null   float64
dtypes: float64(3), object(4)
memory usage: 67.8+ KB


In [ ]:
# Determining profit
gross_budget["profit"] = gross_budget["worldwide_gross"] - gross_budget["production_budget"]

In [20]:
gross_budget['release_date'] = pd.to_datetime(gross_budget['release_date'])

In [22]:
#Checking distribution of my of my numerical columns
gross_budget.describe().T

,count,mean,min,25%,50%,75%,max,std
release_date,1085,2013-11-19 01:39:32.350230272,1967-09-18 00:00:00,2011-09-02 00:00:00,2013-11-01 00:00:00,2016-02-19 00:00:00,2018-12-25 00:00:00,NaN
production_budget,1085.0,52524546.221198,100000.0,13500000.0,30000000.0,68000000.0,410600000.0,57788606.329374
domestic_gross,1085.0,68696288.647926,0.0,15024049.0,39322544.0,84752907.0,700059566.0,87921537.08385
worldwide_gross,1085.0,173052234.267281,0.0,31149251.0,81424988.0,206598789.0,2048134200.0,245087803.234854


In [23]:
gross_budget.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1085 entries, 0 to 1244
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   original_title     1085 non-null   object        
 1   studio             1085 non-null   object        
 2   foreign_gross      1085 non-null   object        
 3   release_date       1085 non-null   datetime64[ns]
 4   production_budget  1085 non-null   float64       
 5   domestic_gross     1085 non-null   float64       
 6   worldwide_gross    1085 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 67.8+ KB


In [24]:
# save the cleaned data
gross_budget.to_csv('cleaned_gross_budget.csv', index=False)  # Saves without row index

Creation of a cleaned merged data.